# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
# Mathematical computation and data manipulation libraries
import pandas as pd
import numpy as np

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Statistics library
from scipy import stats
from scipy.stats import chi2_contingency

In [2]:
data = pd.read_csv("homepage_actions.csv")
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [8]:
action_summary = data.groupby('id')['action'].nunique()

# Counting users who viewed and clicked
both_viewed_and_clicked = (action_summary == 2).sum()
print(f"Users who viewed and clicked: {both_viewed_and_clicked}")

Users who viewed and clicked: 1860


In [9]:
# Listing all user IDs who clicked
clicked_ids = data[data['action'] == 'click']['id'].unique()

# Listing all user IDs who viewed
viewed_ids = data[data['action'] == 'view']['id'].unique()

# Finding those who clicked without viewing
clicked_without_view = set(clicked_ids) - set(viewed_ids)
print(f"Users who clicked without viewing: {len(clicked_without_view)}")


Users who clicked without viewing: 0


In [10]:
group_overlap = data.groupby('id')['group'].nunique()
overlap_count = (group_overlap > 1).sum()

print(f"Users in both control and experiment groups: {overlap_count}")

Users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### Type of Test

We will conduct an A/B Test using a Chi-Square Test for Independence.

###  Hypotheses for A/B Test

**Null Hypothesis (H₀):**
The click rate is the **same** for both the control and experimental homepage.

 `H₀: p_control = p_experiment`

**Alternative Hypothesis (H₁):**
The click rate is **different** between the control and experimental homepage.

 `H₁: p_control ≠ p_experiment`


### Test

In [14]:
import pandas as pd
from scipy.stats import chi2_contingency

contingency = pd.crosstab(data['group'], data['action'])

contingency = contingency[['click', 'view']]  

contingency['no_click'] = contingency['view'] - contingency['click']
contingency = contingency[['click', 'no_click']]

# Chi-square test
chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-square statistic:", chi2)
print("p-value:", p)

# Interpretation
if p < 0.05:
    print("Reject H₀: There is a significant difference in click-through rates.")
else:
    print("Fail to reject H₀: No significant difference detected.")


Chi-square statistic: 6.712921132285344
p-value: 0.00957168049704227
Reject H₀: There is a significant difference in click-through rates.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [15]:
data['clicked'] = (data['action'] == 'click').astype(int)
binary_df = data.groupby('id').agg({
    'group': 'first',              
    'clicked': 'max'               
}).reset_index()

binary_df.head()

,id,group,clicked
0,182988,experiment,0
1,182994,control,1
2,183089,control,0
3,183136,experiment,0
4,183141,experiment,1


### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [23]:
#Your code here
group_sizes = binary_df['group'].value_counts()
ctr_control = binary_df[binary_df['group'] == 'control']['clicked'].mean()
n_exp = group_sizes['experiment']
expected_clicks_exp = ctr_control * n_exp
print(f"Expected number of clicks in experiment group: {expected_clicks_exp:.2f}")

Expected number of clicks in experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [29]:
#Your code here
actual_clicks_exp = binary_df[(binary_df['group'] == 'experiment') & (binary_df['clicked'] == 1)].shape[0]

std_dev = np.sqrt(n_exp * ctr_control * (1 - ctr_control))

z_score = (actual_clicks_exp - expected_clicks_exp) / std_dev
print(f"Z-score: {z_score:.4f}")

Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [30]:
#Your code here
from scipy.stats import norm
p_value = 2 * (1 - norm.cdf(abs(z_score)))
print(f"P-value: {p_value:.5f}")

alpha = 0.05
if p_value < alpha:
    print("Reject H₀: There is a statistically significant difference in click rates.")
else:
    print("Fail to reject H₀: No statistically significant difference detected.")

P-value: 0.00025
Reject H₀: There is a statistically significant difference in click rates.


### Analysis:

**Does this result roughly match that of the previous statistical test?**

Both the Chi-square test and the z-test lead to the same conclusion: there is a statistically significant increase in click rate for the experiment group compared to the control group.

While the Chi-square test yielded a p_value of 0.0096 and the z-test a p_value of 0.00025, both are well below the 0.05 significance level. There slight difference in p-values but the overall decision remains the same.
In conclusion, the experimental homepage appears to be significantly more effective at generating clicks.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.